In [1]:
import pandas as pd
from numpy import nan
import category_encoders as ce
pd.set_option('display.max_columns', 500)

be = ce.BinaryEncoder()

df = pd.read_csv('data.csv', low_memory=False)

print('target: ', df.y.dtypes)
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] File b'data.csv' does not exist: b'data.csv'

In [2]:
# df.isna().sum()

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from fancyimpute import SimpleFill, IterativeImputer, KNN
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score as cvs
X_y = df#.sample(20000)
X = X_y.drop('y', axis=1)
y = X_y.y

Using TensorFlow backend.


In [4]:
pipe = Pipeline(steps = [('impute', SimpleFill(fill_method="mean")), ('scaler', StandardScaler()), ('linear', Ridge())])

pipe.fit(X, y)

ValueError: Input contains infinity or a value too large for dtype('float64').

In [74]:
from sklearn.base import BaseEstimator

?SimpleFill

# titanic 

- make demon, remove data
- make function that takes a df and a list of imputers, returns a df of coefs or feature importances. 

In [2]:
from typing import List
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from fancyimpute import SimpleFill, IterativeImputer, KNN
from sklearn.ensemble import RandomForestRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score as cvs
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from category_encoders import BinaryEncoder
from tqdm import tqdm
from scipy.stats import bernoulli




titanic_ = pd.read_csv('titanic-train.csv')

be = BinaryEncoder()
titanic = be.fit_transform(titanic_.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Survived'], axis=1))

survived = titanic_.Survived




def mk_regressions(imputers: List[BaseEstimator]) -> List[Pipeline]: 
    return [Pipeline(steps = [('impute', imputer), ('regress', Ridge())]) for imputer in imputers]

def mk_ensembles(imputers: List[BaseEstimator]) -> List[Pipeline]:
    return [Pipeline(steps = [('impute', imputer), ('tree', RandomForestRegressor())]) for imputer in imputers]

def demon(dat: pd.DataFrame, ratio: float) -> pd.DataFrame: 
    def ident_or_nan(x: float) -> float: 
        coin = bernoulli(ratio)
        if coin.rvs()==1: 
            return nan
        else: 
            return x
    
    return dat.assign(**{feat: [ident_or_nan(x) for x in dat[feat].values] for feat in dat.columns})



def mk_df(estimators: List[Pipeline], ratio: float, passes: int = 10) -> pd.DataFrame: 

    weights = {}
    
    # control
    for lm in regressions:
        label_ = 'control_'+lm.steps[0][1].__str__().split('(')[0]
        lm.fit(titanic, survived)
        weights[label_] = lm.named_steps[lm.steps[1][0]].coef_
        np.append(weights[label_], [lm.steps[0][1].__str__().split('(')[0]])
    
    for k in tqdm(range(passes), desc='fitting three models EACH  '): 
        
        for lm in regressions: 
            lm.fit(demon(titanic, ratio=ratio), survived)
            weights[k] = lm.named_steps[lm.steps[1][0]].coef_
            np.append(weights[k], [lm.steps[0][1].__str__().split('(')[0]])
    
    return (pd.DataFrame(weights)
            .rename(columns={'KNN(fill_meth': 'KNN'})
            .T
            .rename(columns={k: f'coef_{k+1}' 
                             for k 
                             in range(titanic.shape[1])}))

regressions = mk_regressions([SimpleImputer(strategy="mean"), IterativeImputer(), KNN(k=3, verbose=False)])

regressor_df = mk_df(regressions, 0.3)

# trees = mk_ensembles([SimpleImputer(strategy="mean"), IterativeImputer(), KNN(k=3, verbose=False)])

# trees_df = mk_df(trees)

# trees_df.boxplot()
fig, ax = plt.subplots()
ax.set_title('spread of coefs with 30% missing')
regressor_df.boxplot(grid=False, ax=ax);

Using TensorFlow backend.
fitting three models EACH  :  10%|█         | 1/10 [00:23<03:27, 23.11s/it]

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.stats import bernoulli




def demon(dat: pd.DataFrame, ratio: float) -> pd.DataFrame: 
    def ident_or_nan(x: float) -> float: 
        coin = bernoulli(ratio)
        if coin.rvs()==1: 
            return nan
        else: 
            return x
    
    return dat.assign(**{feat: [ident_or_nan(x) for x in dat[feat].values] for feat in dat.columns})

demon(titanic, 0.1).isna().sum()

In [154]:
demon

Pclass          0
Sex_0           0
Sex_1           0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_0      0
Embarked_1      0
Embarked_2      0
dtype: int64

In [157]:
?KNN

Init signature:
KNN(
    k=5,
    orientation='rows',
    use_argpartition=False,
    print_interval=100,
    min_value=None,
    max_value=None,
    normalizer=None,
    verbose=True,
)
Docstring:     
k-Nearest Neighbors imputation for arrays with missing data.
Works only on dense arrays with at most a few thousand rows.

Assumes that each feature has been centered and rescaled to have
mean 0 and variance 1.

Inspired by the implementation of kNNImpute from the R package
imputation.
See here:
https://www.rdocumentation.org/packages/imputation/versions/2.0.3/topics/kNNImpute
Init docstring:
Parameters
----------
k : int
    Number of neighboring rows to use for imputation.

orientation : str
    Which axis of the input matrix should be treated as a sample
    (default is "rows" but can also be "columns")

use_argpartition : bool
   Use a more naive implementation of kNN imputation whichs calls
   numpy.argpartition for each row/column pair. May give NaN if fewer
   than k neighbors ar

In [1]:
15 * 30

450

In [3]:

titanic_ = pd.read_csv('titanic-train.csv')

be = BinaryEncoder()
titanic = be.fit_transform(titanic_.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'Survived'], axis=1))

survived = titanic_.Survived


titanic.isna().sum()

Pclass          0
Sex_0           0
Sex_1           0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_0      0
Embarked_1      0
Embarked_2      0
dtype: int64

In [14]:
print("missing rate by feature: ")
(titanic_.isna().sum() / titanic_.shape[0]).map(lambda f: f"{f*100:.4} %")

missing rate by feature: 


PassengerId       0.0 %
Survived          0.0 %
Pclass            0.0 %
Name              0.0 %
Sex               0.0 %
Age             19.87 %
SibSp             0.0 %
Parch             0.0 %
Ticket            0.0 %
Fare              0.0 %
Cabin            77.1 %
Embarked       0.2245 %
dtype: object

In [20]:
titanic_.isna().corr()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pclass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.144111,-0.023616
SibSp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ticket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
titanic_.fillna('mean')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,mean,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,mean,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,mean,S
5,6,0,3,"Moran, Mr. James",male,mean,0,0,330877,8.4583,mean,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,mean,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,mean,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,mean,C
